# Лабораторная работа 2

## Постановка задачи

**Часть 1: «Следствие ведёт детектив»**

- Возьмите датасет из задания №1 
- Проанализируйте пропуски: определите их процент и расположение (случайны ли пропуски или есть закономерность?)
- Сформулируйте гипотезу о возможных причинах появления пропусков.
- Проанализируйте датасет на выбросы удобным вам методом.

---

**Часть 2: «Выбор подходящего оружия»**

- Примените несколько разных методов работы с пропусками, выбросами.
- Приведите данные в единый вид, поработайте над категориальными признаками.
- Оцените, как каждый из методов повлиял на распределение данных и результаты простого анализа (например, расчёт среднего или простая регрессия).

---

**Часть 3: «Раскрытие дела»**

- Выберите и обоснуйте самый эффективный метод для вашего набора данных.
- Сформулируйте краткую рекомендацию по оптимальному подходу к обработке пропущенных значений в вашем датасете.

🏆 **Дополнительные баллы** («Расследование с изюминкой»):

*+3 балла* Предложите собственный метод или комбинацию методов, которые лучше всего подходят именно для вашего случая. именно для вашего случая.

## Описание датасета

**Идея:**

Получить данные о продуктах и ценах на них в разных магазинах, для возможности поиска наиболее выгодных предложений.
Также предполагается извлечение дополнительных признаков из описания товаров и поддержка гибкого добавления новых источников (магазинов).
Производится веб-скрейпинг данных с сайтов https://magnit.ru/ (Магнит) и https://5ka.ru/ (Пятерочка). В данной работе производится сбор данных о ценах на товары категории "Сыр", за продолжительное время (в идеале, за каждый день).

## Импорт библиотек

In [156]:
!pip install -q ydata_profiling psycopg2 matplotlib seaborn polars scikit-learn fancyimpute rapidfuzz

In [157]:
from sqlalchemy import create_engine, Column, Float, String, Date, Boolean, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

from ydata_profiling import ProfileReport
from typing import Optional, Tuple, Dict
from sklearn.impute import SimpleImputer
from fancyimpute import IterativeImputer

import matplotlib.pyplot as plt
from rapidfuzz import fuzz
from datetime import date
import seaborn as sns
import pandas as pd
import polars as pl
import numpy as np
import requests
import random
import time
import csv
import os
import re

## Конфигурация подключения к базе данных

In [159]:
Base = declarative_base()

DATABASE_URL = "postgresql+psycopg2://{username}:{password}@{host}:{port}/{db_name}".format(
    host=os.getenv("POSTGRES_HOST", "localhost"),
    port=os.getenv("POSTGRES_PORT", "5432"),
    db_name=os.getenv("POSTGRES_DB", "store_parser"),
    username=os.getenv("POSTGRES_USER", "postgres"),
    password=os.getenv("POSTGRES_PASSWORD", "123456"),

)

def init_db():
    engine = create_engine(DATABASE_URL)
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    return Session()

C:\Users\admin\AppData\Local\Temp\ipykernel_9588\988954521.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Класс, соответствующий структуре таблицы в БД

In [161]:
class Product(Base):
    __tablename__ = 'store_products'

    name = Column(String, nullable=False, primary_key=True)
    brand = Column(String)
    price = Column(Float)  
    weight = Column(Float)
    unit = Column(String)
    fat_content = Column(Float, nullable=True)
    is_sliced = Column(Boolean, nullable=True)
    is_bzmj = Column(Boolean, nullable=True)
    is_creamy = Column(Boolean, nullable=True)
    discount = Column(Float, nullable=True) 
    rating = Column(Float, nullable=True)
    date_added = Column(Date, default=date.today, primary_key=True)  
    store = Column(String, nullable=False, primary_key=True)

    __table_args__ = {'extend_existing': True}

    def __repr__(self):
        return f"<Product(name='{self.name}', price={self.price}, date_added={self.date_added}, store={self.store})>"

## Инициализация БД

In [163]:
session = init_db()

In [164]:
# Определение схемы данных
load_schema = {
    "name": pl.Utf8,
    "price": pl.Float64,
    "date_added": pl.Date,
    "store": pl.Utf8,
    "brand": pl.Utf8,
    "weight": pl.Float32,
    "unit": pl.Utf8,
    "fat_content": pl.Float32,
    "is_sliced": pl.Boolean,
    "is_bzmj": pl.Boolean,
    "is_creamy": pl.Boolean,
    "discount": pl.Float32,
    "rating": pl.Float32
}

## Получение всех данных

In [166]:
results = (
    session
    .query(Product)
    .order_by(Product.date_added.asc())
    .all()
)

data = [{
    "name": product.name,
    "price": product.price,
    "date_added": product.date_added,
    "store": product.store,
    "brand": product.brand,
    "weight": product.weight,
    "unit": product.unit,
    "fat_content": product.fat_content,
    "is_sliced": product.is_sliced,
    "is_creamy": product.is_creamy,
    "is_bzmj": product.is_bzmj,
    "discount": product.discount,
    "rating": product.rating
} for product in results]

# df = pd.DataFrame(data)
df = pl.DataFrame(data, schema=load_schema)
df.sample(5)

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Поставы Городок Пармезан гранд 45% 200г""",179.99,2025-03-26,"""Магнит""","""Поставы Городок Пармезан""",200.0,"""г""",45.0,null,null,null,null,null
"""Сыр Hochland Сливочный плавленый 55% БЗМЖ 200г""",175.99,2025-04-03,"""Пятерочка""","""Hochland Сливочный""",200.0,"""г""",55.0,null,true,true,20.0,4.94
"""Сыр Viola Бутербродный нарезка 45% 120г""",149.99,2025-04-04,"""Магнит""","""Viola Бутербродный""",120.0,"""г""",45.0,true,null,null,16.0,4.7
"""Сыр Поставы Городок Пармезан гранд 45% 200г""",179.99,2025-03-29,"""Магнит""","""Поставы Городок Пармезан""",200.0,"""г""",45.0,null,null,null,10.0,4.5
"""Сыр Pretto Маскарпоне 80% 250г""",279.99,2025-04-03,"""Магнит""","""Pretto Маскарпоне""",250.0,"""г""",80.0,null,null,null,30.0,4.8


In [167]:
session.close()

## Анализ данных

In [169]:
df.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         1572 non-null   object        
 1   price        1572 non-null   float64       
 2   date_added   1572 non-null   datetime64[ms]
 3   store        1572 non-null   object        
 4   brand        1572 non-null   object        
 5   weight       1572 non-null   float32       
 6   unit         1572 non-null   object        
 7   fat_content  1512 non-null   float32       
 8   is_sliced    181 non-null    object        
 9   is_bzmj      880 non-null    object        
 10  is_creamy    527 non-null    object        
 11  discount     739 non-null    float32       
 12  rating       1151 non-null   float32       
dtypes: datetime64[ms](1), float32(4), float64(1), object(7)
memory usage: 135.2+ KB


### Отчет о качестве данных от YData

In [171]:
profile = ProfileReport(df.to_pandas(), title="Data Profiling Report", explorative=True)

# Сохраняем отчет в HTML файл
profile.to_file("data_profile_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 53.40it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Заполнение пропусков bool-значений

#### Поле is_sliced

Предположим, что могут быть другие подстроки, определяющие флаг того, что сыр в нарезке. Попробуем найти подстроки "%рез%", "%ломт%", "%слайс%". Для остальных по логике проставим false вместо null.

In [175]:
sliced_sub = ['рез', 'ломт', 'слайс']

In [176]:
sliced = (
    df
    .filter(
        ~ pl.col('name').str.contains('нарезка')
        & pl.col('name').str.contains_any(sliced_sub)
    )
    .select('name', 'store', 'is_sliced')
    .unique()
)
pl.Config.set_fmt_str_lengths(100)
sliced

name,store,is_sliced
str,str,bool
"""Сыр Hochland Гриль Чиз плавленый ломтики 40% БЗМЖ 150г""","""Пятерочка""",null
"""Сыр плавленый Hochland Чизбургер ломтики 45% 150г""","""Магнит""",null
"""Сыр плавленый Hochland Сэндвич ломтики 45% БЗМЖ 150г""","""Пятерочка""",null
"""Сыр Сыробогатов Для бургера плавленый слайсы 25% БЗМЖ 112г""","""Пятерочка""",null


In [177]:
df = df.with_columns(
    pl.when(
        ~pl.col("name").str.contains("нарезка") &
        pl.col("name").str.contains_any(sliced_sub)
    )
    .then(True)
    .otherwise(False)
    .alias("is_sliced")
)

#### Поле is_bzmj

Задача не легкая, поскольку в магазине Магнит принципиально нигде не указывают этот признак. Можно попробовать по смэтченным продуктам проставить аналогично "пятерочным", а остальным проставить false, грубая прикидка.

In [180]:
df_copy = df.to_pandas().copy()

In [181]:
df_copy['name_normalized'] = df_copy['name'].str.lower()

# Функция для сравнения строк с использованием rapidfuzz
def is_similar(str1, str2, threshold=60):
    return fuzz.ratio(str1, str2) >= threshold

# Группировка товаров по схожести наименований, fat_content и weight
def group_similar_products(df, threshold=80):
    groups = []  # Список для хранения групп
    group_ids = {}  # Словарь для хранения id групп
    next_id = 1  # Счетчик для уникальных id

    for i, row in df.iterrows():
        matched_group = None
        for group in groups:
            # Проверяем совпадение fat_content и weight
            if (row['fat_content'] == group['fat_content'] and
                row['weight'] == group['weight'] and
                row['brand'] == group['brand'] and
                is_similar(row['name_normalized'], group['name_normalized'], threshold)):
                matched_group = group
                break

        if matched_group:
            group_ids[i] = matched_group['id']  # Присваиваем id существующей группы
        else:
            # Создаем новую группу
            new_group = {
                'id': next_id, 
                'name_normalized': row['name_normalized'],
                'fat_content': row['fat_content'], 
                'brand': row['brand'],
                'weight': row['weight']
            }
            groups.append(new_group)
            group_ids[i] = next_id
            next_id += 1

    df['product_id'] = df.index.map(group_ids)
    return df

df_with_ids = group_similar_products(df_copy)

In [182]:
selected = (
    df_with_ids
    .groupby('product_id', as_index=False)
    .agg({
        'name': lambda x: list(x.unique()),
        'is_bzmj': lambda x: list(x.unique())
    })
    # Добавляем флаги для проверки уникальности
    .assign(
        is_single_bzmj=lambda x: x['is_bzmj'].apply(len) == 1,
        contains_true=lambda x: x['is_bzmj'].apply(lambda lst: True in lst) 
    )
    # Фильтруем по условию (name ИЛИ is_bzmj имеют одно значение)
    .query('not is_single_bzmj and contains_true')
    .drop(columns=['is_single_bzmj', 'contains_true'])
)

pd.set_option('display.max_colwidth', None)
selected.head(50)

,product_id,name,is_bzmj
3,4,"[Сыр Брест-Литовск Российский 50% 200г, Сыр полутвердый Брест-Литовск Российский 50% БЗМЖ 200г]","[None, True]"
19,20,"[Сыр творожный Hochland с зеленью 60% 140г, Сыр творожный Hochland с чесноком 60% БЗМЖ 140г, Сыр творожный Hochland сливочный 60% 140г]","[None, True]"
95,96,"[Сыр Natura Сливочный 45% 300г, Сыр Natura Сливочный нарезка 45% БЗМЖ 300г]","[None, True]"
100,101,"[Сыр Ларец с грецкими орехами 45% БЗМЖ 245г, Сыр Ларец с грецкими орехами 45% 245г]","[True, None]"
101,102,"[Сыр Фетакса без рассола 45% 200г, Сыр Фетакса без рассола 45% БЗМЖ 200г]","[None, True]"
150,151,"[Сыр Viola Бутербродный нарезка 45% 120г, Сыр Viola Бутербродный полутвердый нарезка 45% БЗМЖ 120г]","[None, True]"
165,166,"[Сыр Брест-Литовск Классический 45% нарезка 150г, Сыр Брест-Литовск Классический нарезка 45% БЗМЖ 150г]","[None, True]"


In [183]:
# Развернуть списки в selected и получить уникальные имена
target_names = selected.explode('name')['name'].unique().tolist()
target_names

['Сыр Брест-Литовск Российский 50% 200г',
 'Сыр полутвердый Брест-Литовск Российский 50% БЗМЖ 200г',
 'Сыр творожный Hochland с зеленью 60% 140г',
 'Сыр творожный Hochland с чесноком 60% БЗМЖ 140г',
 'Сыр творожный Hochland сливочный 60% 140г',
 'Сыр Natura Сливочный 45% 300г',
 'Сыр Natura Сливочный нарезка 45% БЗМЖ 300г',
 'Сыр Ларец с грецкими орехами 45% БЗМЖ 245г',
 'Сыр Ларец с грецкими орехами 45% 245г',
 'Сыр Фетакса без рассола 45% 200г',
 'Сыр Фетакса без рассола 45% БЗМЖ 200г',
 'Сыр Viola Бутербродный нарезка 45% 120г',
 'Сыр Viola Бутербродный полутвердый нарезка 45% БЗМЖ 120г',
 'Сыр Брест-Литовск Классический 45% нарезка 150г',
 'Сыр Брест-Литовск Классический нарезка 45% БЗМЖ 150г']

In [184]:
# Проставить True для всех записей с name из target_names

df = df.with_columns(
    pl.when(pl.col('name').is_in(target_names))
    .then(True)  # Условие 3: имя в списке → True
    .when(pl.col('is_bzmj').is_null()) 
    .then(False)  # Условие 2: None и имя не в списке → False
    .otherwise(pl.col('is_bzmj'))  # Условие 1: сохраняем текущее значение
    .alias('is_bzmj')
)
df.head()

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Экомилк творожный 60% 400г""",269.99,2025-03-26,"""Магнит""","""Экомилк""",400.0,"""г""",60.0,false,false,true,null,null
"""Сыр Свежий Ряд Чечил спагетти копченый БЗМЖ 100г""",149.99,2025-03-26,"""Пятерочка""","""Свежий Ряд Чечил""",100.0,"""г""",null,false,true,null,null,null
"""Сыр Montarell Бри мягкий 60% БЗМЖ 125г""",207.99,2025-03-26,"""Пятерочка""","""Montarell Бри""",125.0,"""г""",60.0,false,true,null,null,null
"""Сыр Брест-Литовск Российский 50% 200г""",239.99,2025-03-26,"""Магнит""","""Брест-Литовск Российский""",200.0,"""г""",50.0,false,true,null,null,null
"""Сыр плавленый Карат Шоколадный 30% 230г""",159.99,2025-03-26,"""Магнит""","""Карат Шоколадный""",230.0,"""г""",30.0,false,false,true,null,null


In [185]:
(
    df
    .filter(pl.col('is_bzmj') == True)
    .filter(pl.col('store') == 'Магнит') # в наимнованиях Магнита нет указания БЗМЖ, проверим работу заполнения пропусков
).head()

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Брест-Литовск Российский 50% 200г""",239.99,2025-03-26,"""Магнит""","""Брест-Литовск Российский""",200.0,"""г""",50.0,false,true,null,null,null
"""Сыр творожный Hochland с зеленью 60% 140г""",119.99,2025-03-26,"""Магнит""","""Hochland""",140.0,"""г""",60.0,false,true,true,null,null
"""Сыр Natura Сливочный 45% 300г""",319.99,2025-03-26,"""Магнит""","""Natura Сливочный""",300.0,"""г""",45.0,false,true,null,null,null
"""Сыр Фетакса без рассола 45% 200г""",199.99,2025-03-26,"""Магнит""","""Фетакса""",200.0,"""г""",45.0,false,true,null,null,null
"""Сыр Viola Бутербродный нарезка 45% 120г""",149.99,2025-03-26,"""Магнит""","""Viola Бутербродный""",120.0,"""г""",45.0,false,true,null,null,null


#### Поле is_creamy

Можно аналогично is_bzmj проверить

In [188]:
selected = (
    df_with_ids
    .groupby('product_id', as_index=False)
    .agg({
        'name': lambda x: list(x.unique()),
        'is_creamy': lambda x: list(x.unique())
    })
    # Добавляем флаги для проверки уникальности
    .assign(
        is_single_creamy=lambda x: x['is_creamy'].apply(len) == 1,
        contains_true=lambda x: x['is_creamy'].apply(lambda lst: True in lst) 
    )
    # Фильтруем по условию (name ИЛИ is_bzmj имеют одно значение)
    .query('not is_single_creamy and contains_true')
    .drop(columns=['is_single_creamy', 'contains_true'])
)

pd.set_option('display.max_colwidth', None)
selected.head(50)

,product_id,name,is_creamy
98,99,[Плавленый продукт Дружба 50% 100г],"[None, True]"
171,172,[Плавленый продукт Рязанский ЗПС колбасный копченый 45% 300г],"[None, True]"


In [189]:
# Развернуть списки в selected и получить уникальные имена
target_names = selected.explode('name')['name'].unique().tolist()
target_names

['Плавленый продукт Дружба 50% 100г',
 'Плавленый продукт Рязанский ЗПС колбасный копченый 45% 300г']

In [190]:
# Проставить True для всех записей с name из target_names

df = df.with_columns(
    pl.when(pl.col('name').is_in(target_names))
    .then(True)  # Условие 3: имя в списке → True
    .when(pl.col('is_creamy').is_null()) 
    .then(False)  # Условие 2: None и имя не в списке → False
    .otherwise(pl.col('is_creamy'))  # Условие 1: сохраняем текущее значение
    .alias('is_creamy')
)
df.head()

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Экомилк творожный 60% 400г""",269.99,2025-03-26,"""Магнит""","""Экомилк""",400.0,"""г""",60.0,false,false,true,null,null
"""Сыр Свежий Ряд Чечил спагетти копченый БЗМЖ 100г""",149.99,2025-03-26,"""Пятерочка""","""Свежий Ряд Чечил""",100.0,"""г""",null,false,true,false,null,null
"""Сыр Montarell Бри мягкий 60% БЗМЖ 125г""",207.99,2025-03-26,"""Пятерочка""","""Montarell Бри""",125.0,"""г""",60.0,false,true,false,null,null
"""Сыр Брест-Литовск Российский 50% 200г""",239.99,2025-03-26,"""Магнит""","""Брест-Литовск Российский""",200.0,"""г""",50.0,false,true,false,null,null
"""Сыр плавленый Карат Шоколадный 30% 230г""",159.99,2025-03-26,"""Магнит""","""Карат Шоколадный""",230.0,"""г""",30.0,false,false,true,null,null


#### Поле discount

При сборе данных было 2 момента:
- признак не с первого дня стал вычитываться с источника
- нет заполнения пустых значений нулями при сборе данных, необходимо изменение в алгоритме сбора
- отследим день, в который стали фиксироваться значения признака

In [193]:
(
    df
    .filter(~ pl.col('discount').is_null())
    .sort('date_added')
    .head(1)
)

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Поставы Городок Пармезан гранд 45% 200г""",179.99,2025-03-29,"""Магнит""","""Поставы Городок Пармезан""",200.0,"""г""",45.0,false,false,false,10.0,4.5


Алгоритм следующий:
- в дни с 29.03 заполняем null на 0, означает что скидки нет
- до 29.03 справедливее заполнить средней скидкой на товар за время наблюдения

In [195]:
df = df.with_columns(
    pl.when((pl.col('date_added') >= date(2025, 3, 29)) & pl.col('discount').is_null())
    .then(0)
    .when(pl.col('discount').is_null())
    .then(
        df
        .group_by("name")
        .agg(avg_discount=pl.coalesce(pl.col("discount").mean(), 0).alias("avg_discount"))
        .join(df, on="name")["avg_discount"]
        .round(2)
    )
    .otherwise(pl.col('discount'))
    .alias('discount')
)

#### Поле fat_content

Можно попробовать аналогично is_bzmj

In [198]:
selected = (
    df_with_ids
    .groupby('product_id', as_index=False)
    .agg({
        'name': lambda x: list(x.unique()),
        'fat_content': lambda x: list(x.unique())
    })
    # Добавляем флаги для проверки уникальности
    .assign(
        is_single_fat_content=lambda x: x['fat_content'].apply(len) == 1
    )
    .query('not is_single_fat_content')
    .drop(columns=['is_single_fat_content'])
)

pd.set_option('display.max_colwidth', None)
selected.head(50)

,product_id,name,fat_content


Тогда найдем пропуски fat_content

In [200]:
fat_cont_empty = (
    df
    .filter(pl.col('fat_content').is_null())
    ['name']
    .unique()
)
fat_cont_empty

name
str
"""Сыр Premiere of taste Пармезан весовой"""
"""Сыр Liebendorf Гауда полутвердый нарезка БЗМЖ 150г"""
"""Сыр Premiere of taste мягкий с белой плесенью Бри 125г"""
"""Сыр Свежий Ряд Чечил спагетти копченый БЗМЖ 100г"""
"""Сыр Моя цена Российский весовой"""
"""Сыр Premiere of taste мягкий с белой плесенью Камамбер 125г"""
"""Сыр LiebenDorf Маасдам полутвердый нарезка БЗМЖ 150г"""
"""Сыр Тэрэз копченый Балыковый 100г"""


Можно, опираясь на данные из интернета, составить словарик, с которым сопоставлять наименование сыра и подставлять жирность. Но это требует ручного составления словаря по видам сыров - не только по представленным выше, а расширенному списку, т.к. есть вероятность того, что придут новые данные.
ИЛИ считать как медиану между сырами с той же категорией (Выбрано)

In [202]:
categories = [
    "бри", "российск", "камамбер", 
    "копчен", "пармезан", "гауда", 
    "маасдам", "эдам", "тильзитер",
    "бутербродн", "чеддер", "творожн",
    "маскарпоне", "рикотта", "моцарелла",
    "адыгейск", "ламбер", "чечил",
    "монастырск", "сливочн", "голландск",
    "сулугуни", "брынза", "классическ",
    "сливочн", "рассольн", "плавленый",
]

In [203]:
# Шаг 1: Добавляем столбец с категорией для каждой строки
df = df.with_columns(
    pl.coalesce(
        *[
            pl.when(pl.col("name").str.to_lowercase().str.contains(cat, literal=True))
            .then(pl.lit(cat))
            for cat in categories
        ],
        pl.lit(None)  # Если ни одна категория не найдена
    )
    .alias("category") 
)
df.sample(7)

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating,category
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32,str
"""Сыр Брест-Литовск Классический 45% нарезка 150г""",179.99,2025-04-04,"""Магнит""","""Брест-Литовск Классический""",150.0,"""г""",45.0,false,true,false,18.0,4.7,"""классическ"""
"""Сыр Viola Маасдам полутвердый 45% БЗМЖ 180г""",237.99,2025-03-28,"""Пятерочка""","""Viola Маасдам""",180.0,"""г""",45.0,false,true,false,15.0,null,"""маасдам"""
"""Сыр Liebendorf с грибами плавленый 55% БЗМЖ 400г""",257.59,2025-04-04,"""Пятерочка""","""Liebendorf""",400.0,"""г""",55.0,false,true,true,0.0,4.88,"""плавленый"""
"""Сыр Ламбер 50% весовой""",324.97,2025-03-26,"""Магнит""","""Ламбер""",1.0,"""кг""",50.0,false,false,false,18.0,null,"""ламбер"""
"""Пармезан Trattoria di Maestro Turatti твердый 40% БЗМЖ""",159.99,2025-04-02,"""Пятерочка""","""Trattoria""",1.0,"""кг""",40.0,false,true,false,0.0,4.89,"""пармезан"""
"""Сыр Радость вкуса Топленочка с фенугреком 45% БЗМЖ 180г""",299.99,2025-04-03,"""Пятерочка""","""Радость""",180.0,"""г""",45.0,false,true,false,0.0,4.84,null
"""Сыр Белебеевский 45% 190г""",194.99,2025-04-03,"""Магнит""","""Белебеевский""",190.0,"""г""",45.0,false,false,false,18.0,4.7,null


In [204]:
# Шаг 2: Вычисляем медианы по категориям
median_values = (
    df
    .filter(pl.col("category").is_not_null())  # Игнорируем строки без категории
    .group_by("category")
    .agg(pl.median("fat_content").alias("median_fat"))
)
median_values

category,median_fat
str,f32
"""брынза""",45.0
"""тильзитер""",45.0
"""бутербродн""",45.0
"""сулугуни""",45.0
"""монастырск""",45.0
…,…
"""маскарпоне""",80.0
"""эдам""",45.0
"""пармезан""",40.0


In [205]:
# Шаг 3: Заполняем пропуски
df = (
    df.join(median_values, on="category", how="left")
    .with_columns(
        pl.coalesce(pl.col("fat_content"), pl.col("median_fat"), 50).alias("fat_content") # Если не найдена категория, ставим среднее как 50
    )
    .drop("median_fat", "category")  # Удаляем вспомогательные столбцы
)

df.sample(5)

name,price,date_added,store,brand,weight,unit,fat_content,is_sliced,is_bzmj,is_creamy,discount,rating
str,f64,date,str,str,f32,str,f32,bool,bool,bool,f32,f32
"""Сыр Premiere of taste Моцарелла 40% 200г""",229.99,2025-04-02,"""Магнит""","""Premiere""",200.0,"""г""",40.0,false,false,false,0.0,4.7
"""Сыр ЭкоНива Momente aus Schollbrunn полутвердый 50% 200г""",199.99,2025-03-26,"""Магнит""","""ЭкоНива Momente""",200.0,"""г""",50.0,false,false,false,13.0,null
"""Сыр творожный Hochland с чесноком 60% БЗМЖ 140г""",103.99,2025-03-31,"""Пятерочка""","""Hochland""",140.0,"""г""",60.0,false,true,true,25.0,4.9
"""Фета Sveza нежная 45% БЗМЖ 250г""",199.99,2025-03-26,"""Пятерочка""","""Sveza""",250.0,"""г""",45.0,false,true,false,15.0,null
"""Сыр Сыробогатов Для бургера плавленый слайсы 25% БЗМЖ 112г""",79.99,2025-03-26,"""Пятерочка""","""Сыробогатов Для""",112.0,"""г""",25.0,true,true,true,33.0,null


#### Поле rating

Аналогично discount, данные до 29.03 не собирались, а позже - null, если у продукта нет оценки. Попробуем сгруппировать по названию и, может быть, доставить значения тем продуктам, у которых есть оценка в записи за другой день

In [208]:
grouped_ratings = (
    df
    .group_by("name")  # Группировка по "name"
    .agg(
        pl.col("rating").unique().alias("unique_ratings")  # Убираем `.list`
    )
    .with_columns(
        (pl.col("unique_ratings").list.len() == 1).alias("is_single_rating")  # Проверка длины списка
    )
    .filter((pl.col('is_single_rating') == False) & pl.col('unique_ratings').list.contains(None))
    .with_columns(
        pl.col("unique_ratings")
        .list.eval(pl.element().fill_null(pl.element().mean())) 
        .list.mean()  # Вычисляем среднее по списку
        .alias("mean_rating")
    )
    .drop("is_single_rating")
)
print(grouped_ratings.shape)
grouped_ratings.sample(7)

(206, 3)


name,unique_ratings,mean_rating
str,list[f32],f32
"""Сыр Endorf Tulyere твердый 50% БЗМЖ 200г""","[null, 4.86]",4.86
"""Сыр LiebenDorf Маасдам полутвердый нарезка БЗМЖ 150г""","[null, 4.89, 4.9]",4.895
"""Плавленый продукт Дружба 50% 100г""","[null, 4.6]",4.6
"""Сыр Брест-Литовск Российский 50% 150г""","[null, 4.7]",4.7
"""Сыр полутвердый ЭкоНива Тильзитер 50% БЗМЖ 400г""","[null, 4.89, 4.9]",4.895
"""Сыр Vitalat Перлини копченый 40% БЗМЖ 100г""","[null, 4.87, 4.89]",4.88
"""Сыр Oldenburger Protein с грецким орехом нарезка 30% БЗМЖ 125г""","[null, 4.86]",4.86


In [209]:
global_mean = df.select(pl.mean("rating")).item()

df = (
    df
    .join(grouped_ratings, on="name", how="left")
    .with_columns(
        pl.coalesce(pl.col("rating"), pl.col("mean_rating"), global_mean).alias("rating")
    )
    .drop("mean_rating", "unique_ratings")
)

In [210]:
df.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1572 entries, 0 to 1571
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   name         1572 non-null   object        
 1   price        1572 non-null   float64       
 2   date_added   1572 non-null   datetime64[ms]
 3   store        1572 non-null   object        
 4   brand        1572 non-null   object        
 5   weight       1572 non-null   float32       
 6   unit         1572 non-null   object        
 7   fat_content  1572 non-null   float32       
 8   is_sliced    1572 non-null   bool          
 9   is_bzmj      1572 non-null   bool          
 10  is_creamy    1572 non-null   bool          
 11  discount     1572 non-null   float32       
 12  rating       1572 non-null   float32       
dtypes: bool(3), datetime64[ms](1), float32(4), float64(1), object(4)
memory usage: 103.0+ KB


## Выводы

Ввиду специфичности данных и досконального знания о значениях данных и причинах пропусков, были использованы не все возможные методы заполнения пропусков и нормализации данных.
В целом, можно определить несколько преимущественных способов:
- заполнение медианой (в случае заполнения жирности по категории)
- заполнение средним в случае отсутствия значений рейтинга за ранние даты
- мэтчинг одинаковых продуктов с отличием в наименовании и заполнение пропусков согласно имеющимся данным в других записях